In [1]:
import sqlite3
import pandas as pd


In [2]:
# conectar ao arquivo .sqlite
con = sqlite3.connect('db_olist.sqlite')
cur = con.cursor()

In [3]:
# LISTAR AS TABELAS PRESENTES NO BANCO DE DADOS

cur.execute("""
    SELECT name FROM sqlite_master  
    WHERE type= 'table'
""")

print(cur.fetchall())

con.close()

[('customer',), ('geolocation',), ('order_items',), ('order_payments',), ('order_reviews',), ('orders',), ('products',), ('sellers',), ('product_category_name',)]


**Sobre encerrar a conexão a cada operação**

- É uma boa prática liberar os recursos que você não precisa mais. Normalmente, as conexões de banco de dados são muito "caras" e é definitivamente recomendado abrir a conexão, fazer a consulta real e fechar a conexão logo em seguida.

- Porém, como o objetivo desse projeto é para estudo da linguagem de consulta SQL. Terminarei a conexão no final do projeto.

In [4]:
# Reconectar 
# conectar ao arquivo .sqlite
con = sqlite3.connect('db_olist.sqlite')
cur = con.cursor()

## **PERGUNTAS DO CEO:**

**1. Qual o número de clientes únicos de todos os estados?**

In [5]:
query = """
SELECT 
    c.customer_state,
    COUNT(DISTINCT c.customer_id) AS clientes
FROM customer c 
GROUP BY c.customer_state
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)

# head mostra as 5 primeiras linhas
df.head()


,customer_state,clientes
0,AC,81
1,AL,413
2,AM,148
3,AP,68
4,BA,3380


**2. Qual o número total de pedidos únicos, a data mínima e máxima de envio, o valor máximo, mínimo e médio do frete dos pedidos acima de R$ 1.100 por cada vendedor?**

In [6]:
query = """
SELECT 
    oi.seller_id,
    COUNT(DISTINCT oi.product_id),
    MIN(oi.shipping_limit_date),
    MAX(oi.shipping_limit_date),
    MAX(oi.freight_value),
    MIN(oi.freight_value),
    AVG(oi.freight_value) 
FROM order_items oi 
WHERE oi.price < 1100
GROUP BY oi.seller_id;
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)

# head mostra as 5 primeiras linhas
df.head()


,seller_id,COUNT(DISTINCT oi.product_id),MIN(oi.shipping_limit_date),MAX(oi.shipping_limit_date),MAX(oi.freight_value),MIN(oi.freight_value),AVG(oi.freight_value)
0,0015a82c2db000af6aaaf3ae2ecb0532,1,2017-10-12 22:24:16,2017-10-24 23:56:20,21.02,21.02,21.020000
1,001cca7ae9ae17fb1caed9dfb1094831,11,2017-02-08 19:06:04,2018-07-19 21:50:17,114.62,14.72,37.046611
2,001e6ad469a905060d959994f1b41e4f,1,2018-08-13 23:05:23,2018-08-13 23:05:23,17.94,17.94,17.940000
3,002100f778ceb8431b7a1020ff7ab48f,24,2017-09-20 01:10:15,2018-04-18 13:11:45,34.15,4.91,14.430182
4,003554e2dce176b5555353e4f3555ac8,1,2017-12-21 07:11:03,2017-12-21 07:11:03,19.38,19.38,19.380000


**3. Qual o valor médio, máximo e mínimo do preço de todos os pedidos de cada produto?**

In [7]:
query = """
SELECT 
    oi.product_id,
    AVG(oi.price),
    MAX(oi.price),
    MIN(oi.price)
FROM order_items oi 
GROUP BY oi.product_id
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)

# head mostra as 5 primeiras linhas
df.head()


,product_id,AVG(oi.price),MAX(oi.price),MIN(oi.price)
0,00066f42aeeb9f3007548bb9d3f33c38,101.65,101.65,101.65
1,00088930e925c41fd95ebfe695fd2655,129.90,129.90,129.90
2,0009406fd7479715e4bef61dd91f2462,229.00,229.00,229.00
3,000b8f95fcb9e0096488278317764d19,58.90,58.90,58.90
4,000d9be29b5207b54e86aa1b1ac54872,199.00,199.00,199.00


**4. Qual a quantidade de vendedores distintos que receberam algum pedido antes do dia 23 de setembro de 2016 e qual foi o preço médio desses pedidos?**

In [8]:
query = """
SELECT 
    oi.shipping_limit_date,
    COUNT(DISTINCT oi.seller_id) AS vendedores,
    AVG(oi.price) AS avg_price 
FROM order_items oi 
WHERE oi.shipping_limit_date < '2016-09-23 00:00:00'
GROUP BY oi.shipping_limit_date
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)


df


,shipping_limit_date,vendedores,avg_price
0,2016-09-19 00:15:34,1,59.50
1,2016-09-19 23:11:33,1,44.99


**5.  Qual a quantidade de pedidos por tipo de pagamentos?**

In [9]:
query = """
SELECT 
    op.payment_type,
    COUNT(op.order_id) 
FROM order_payments op 
GROUP BY op.payment_type
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df

,payment_type,COUNT(op.order_id)
0,boleto,19784
1,credit_card,76795
2,debit_card,1529
3,not_defined,3
4,voucher,5775


**6. Quantos clientes únicos tiveram seu pedidos com status de “processing”, “shipped” e “delivered”, feitos entre os dias 01 e 31 de Outubro de 2016. Mostrar o resultado somente se o número total de clientes for acima de 5.**

In [10]:
query = """
SELECT 
    o.order_status,
    COUNT(DISTINCT o.order_id) 
FROM orders o 
WHERE o.order_status IN ('processing', 'shipped', 'delivered') AND 
    o.order_purchase_timestamp BETWEEN '2016-10-01' AND '2016-10-31'
GROUP BY o.order_status 
HAVING COUNT(DISTINCT o.order_id) > 5
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df


,order_status,COUNT(DISTINCT o.order_id)
0,delivered,265
1,shipped,8


**7. Quantos produtos estão cadastrados nas categorias: perfumaria, brinquedos, esporte lazer e cama mesa, que possuem entre 5 e 10 fotos, um peso que não está entre 1 e 5 g, um altura maior que 10 cm, uma largura maior que 20 cm. Mostra somente as linhas com mais de 10 produtos únicos.**

In [11]:
query = """
SELECT
    p.product_category_name,
    COUNT(DISTINCT p.product_id) as produtos
FROM products p
WHERE p.product_category_name IN ('perfumaria', 'brinquedos', 'esporte_lazer', 'cama_mesa_banho') 
    AND p.product_photos_qty BETWEEN 5 AND 10 
    AND p.product_weight_g NOT BETWEEN 1 AND 5
    AND p.product_height_cm > 10
    AND p.product_width_cm > 20
GROUP BY p.product_category_name
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df

,product_category_name,produtos
0,brinquedos,68
1,cama_mesa_banho,34
2,esporte_lazer,112
3,perfumaria,8


**8.Quantos produtos estão cadastrados em qualquer categorias que comece com a letra “a” e termine com a letra “o” e que possuem mais de 5 fotos? Mostrar as linhas com mais de 10 produtos.**

In [12]:
query = """
SELECT 
    p.product_category_name,
    COUNT(DISTINCT p.product_id) as produtos
FROM products p 
WHERE p.product_category_name LIKE 'a%o'
    AND p.product_photos_qty > 5
GROUP BY p.product_category_name 
HAVING COUNT(DISTINCT p.product_id) > 10
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df

,product_category_name,produtos
0,automotivo,172


**9. Qual o número de clientes únicos, agrupados por estado e por cidades que comecem com a letra “m”, tem a letra “o” e terminem com a letra “a”? Mostrar os resultados somente para o número de clientes únicos maior que 10.**

In [13]:
query = """
SELECT 
    c.customer_state,
    c.customer_city,
    COUNT(DISTINCT c.customer_id) 
FROM customer c 
WHERE c.customer_city LIKE 'm%o%a'
GROUP BY c.customer_state, c.customer_city 
HAVING COUNT(DISTINCT c.customer_id) > 10
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df

,customer_state,customer_city,COUNT(DISTINCT c.customer_id)
0,MG,matias barbosa,13
1,SP,mairipora,75
2,SP,mococa,34
3,SP,mongagua,42
4,SP,monte azul paulista,19


**10. Qual a quantidade de pedidos com o status em processamento ou cancelada acontecem com a data estimada de entrega maior que 01 de Janeiro de 2017 ou menor que 23 de Novembro de 2016?**

In [14]:
query = """
SELECT 
    o.order_status,
    COUNT(DISTINCT o.order_id)
FROM orders o
WHERE o.order_status = 'processing' OR o.order_status = 'canceled' AND 
    (o.order_approved_at > '2017-01-01' OR o.order_approved_at < '2016-11-23')
GROUP BY o.order_status
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df

,order_status,COUNT(DISTINCT o.order_id)
0,canceled,484
1,processing,301


**11. Gerar uma tabela de dados com 10 linhas, contendo o id do pedido, o id do cliente, o status do pedido, o id do produto e o preço do produto.**

In [15]:
query = """
SELECT 
    o.order_id,
    o.customer_id,
    o.order_status,
    oi.product_id,
    oi.price 
FROM orders o INNER JOIN order_items oi ON (o.order_id = oi.order_id)
LIMIT 10;
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,order_id,customer_id,order_status,product_id,price
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,87285b34884572647811a353c7ac498a,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,595fac2a385ac33a80bd5114aec74eb8,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,aa4383b373c6aca5d8797843e5594415,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,d0b61bfb1de832b15ba9d266ca96e5b0,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,65266b2da20d04dbe00c5c2d3bb7859e,19.90


**12. Gerar uma tabela de dados com 20 linhas, contendo o id do pedido, o estado do cliente, a cidade do cliente, o status do pedido, o id do produto e o preço do produto, somente para clientes do estado de São Paulo**

In [16]:
query = """
SELECT 
    o.order_id,
    c.customer_state,
    c.customer_city, 
    o.order_status,
    oi.product_id,
    oi.price 
FROM orders o INNER JOIN order_items oi ON (o.order_id = oi.order_id)
              INNER JOIN customer c ON (o.customer_id = c.customer_id)
 WHERE c.customer_state = 'SP'
LIMIT 20;
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,order_id,customer_state,customer_city,order_status,product_id,price
0,00e7ee1b050b8499577073aeb2a297a1,SP,franca,delivered,a9516a079e37a9c9c36b9b78b10169e8,124.99
1,29150127e6685892b6eab3eec79f59c7,SP,sao bernardo do campo,delivered,4aa6014eceb682077f9dc4bffebc05b0,289.00
2,b2059ed67ce144a36e2aa97d2c9e9ad2,SP,sao paulo,delivered,bd07b66896d6f1494f5b86251848ced7,139.94
3,951670f92359f4fe4a63112aa7306eba,SP,mogi das cruzes,delivered,a5647c44af977b148e0a3a4751a09e2e,149.94
4,6b7d50bd145f6fc7f33cebabd7e49d0f,SP,campinas,delivered,9391a573abe00141c56e38d84d7d5b3b,230.00


**13. Gerar uma tabela de dados com 50 linhas, contendo o id do pedido, o estado e a cidade do cliente, o status do pedido, o nome da categoria do produto e o preço do produto, somente para pedidos com o status igual a cancelado.**

In [17]:
query = """
SELECT 
    o.order_id,
    c.customer_state,
    c.customer_city, 
    o.order_status,
    p.product_category_name,
    oi.price 
FROM orders o INNER JOIN order_items oi ON (o.order_id = oi.order_id)
              INNER JOIN customer c ON (o.customer_id = c.customer_id)
              INNER JOIN products p ON (oi.product_id = p.product_id)
WHERE o.order_status = 'canceled'
LIMIT 50;
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,order_id,customer_state,customer_city,order_status,product_category_name,price
0,1b9ecfe83cdc259250e1a8aca174f0ad,SP,sao paulo,canceled,informatica_acessorios,25.00
1,714fb133a6730ab81fa1d3c1b2007291,SP,diadema,canceled,moveis_decoracao,69.90
2,3a129877493c8189c59c60eb71d97c29,PR,maringa,canceled,esporte_lazer,35.90
3,0966b61e30c4a07edbd7523f59b3f3e4,RJ,rio das ostras,canceled,moveis_decoracao,68.31
4,0966b61e30c4a07edbd7523f59b3f3e4,RJ,rio das ostras,canceled,moveis_decoracao,68.31


**14.Gerar uma tabela de dados com 80 linhas, contendo o id do pedido, o estado e a cidade do cliente, o status do pedido, o nome da categoria do produto, o preço do produto, a cidade e o estado do vendedor e a data de aprovação do pedido, somente para os pedidos aprovadas a partir do dia 16 de Setembro de 2016.**

In [18]:
query = """
SELECT 
    o.order_id,
    c.customer_state,
    c.customer_city, 
    o.order_status,
    p.product_category_name,
    oi.price,
    s.seller_city,
    s.seller_state,
    o.order_approved_at 
FROM orders o INNER JOIN order_items oi ON (o.order_id = oi.order_id)
              INNER JOIN customer c ON (o.customer_id = c.customer_id)
              INNER JOIN products p ON (oi.product_id = p.product_id)
              INNER JOIN sellers s ON (oi.seller_id = s.seller_id)
WHERE o.order_approved_at > '2016-09-16'
LIMIT 80;
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,order_id,customer_state,customer_city,order_status,product_category_name,price,seller_city,seller_state,order_approved_at
0,e481f51cbdc54678b7cc49136f2d6af7,SP,sao paulo,delivered,utilidades_domesticas,29.99,maua,SP,2017-10-02 11:07:15
1,53cdb2fc8bc7dce0b6741e2150273451,BA,barreiras,delivered,perfumaria,118.70,belo horizonte,SP,2018-07-26 03:24:27
2,47770eb9100c2d0c44946d9cf07ec65d,GO,vianopolis,delivered,automotivo,159.90,guariba,SP,2018-08-08 08:55:23
3,949d5b44dbf5de918fe9c16f97b45f8a,RN,sao goncalo do amarante,delivered,pet_shop,45.00,belo horizonte,MG,2017-11-18 19:45:59
4,ad21c59c0840e6cb83a9ceb5573f8159,SP,santo andre,delivered,papelaria,19.90,mogi das cruzes,SP,2018-02-13 22:20:29


**15. Gerar uma tabela de dados com 70 linhas, contendo o id do pedido, o estado e a cidade do cliente, o status do pedido, o nome da categoria do produto, o preço do produto, a cidade e o estado do vendedor, a data de aprovação do pedido, tipo de pagamento e a nota de avaliação do produto, somente para os pedidos com a nota de avaliação do produto igual a 1.**

In [19]:
query = """
SELECT 
    o.order_id,
    c.customer_state,
    c.customer_city, 
    o.order_status,
    p.product_category_name,
    oi.price,
    s.seller_city,
    s.seller_state,
    o.order_approved_at,
    op.payment_type,
    or2.review_score 
FROM orders o INNER JOIN order_items oi ON (o.order_id = oi.order_id)
              INNER JOIN customer c ON (o.customer_id = c.customer_id)
              INNER JOIN products p ON (oi.product_id = p.product_id)
              INNER JOIN sellers s ON (oi.seller_id = s.seller_id)
              INNER JOIN order_payments op ON (o.order_id = op.order_id)
              INNER JOIN order_reviews or2 ON (o.order_id = or2.order_id)
WHERE or2.review_score = 1
LIMIT 70;
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,order_id,customer_state,customer_city,order_status,product_category_name,price,seller_city,seller_state,order_approved_at,payment_type,review_score
0,b18dcdf73be66366873cd26c5724d1dc,SP,sao paulo,delivered,cama_mesa_banho,119.0,belo horizonte,MG,2018-04-09 20:10:35,credit_card,1
1,b18dcdf73be66366873cd26c5724d1dc,SP,sao paulo,delivered,cama_mesa_banho,99.9,piracicaba,SP,2018-04-09 20:10:35,credit_card,1
2,b18dcdf73be66366873cd26c5724d1dc,SP,sao paulo,delivered,cama_mesa_banho,99.9,piracicaba,SP,2018-04-09 20:10:35,credit_card,1
3,b18dcdf73be66366873cd26c5724d1dc,SP,sao paulo,delivered,cama_mesa_banho,99.9,piracicaba,SP,2018-04-09 20:10:35,credit_card,1
4,583174fbe37d3d5f0d6661be3aad1786,SP,guarulhos,canceled,malas_acessorios,89.9,sao paulo,SP,2018-08-05 19:24:33,credit_card,1


**16. Gerar uma tabela de dados com 20 linhas e contendo as seguintes colunas: 1) Id do pedido, 2) status do pedido, 3) id do produto, 4) categoria do produto, 5) avaliação do pedido, 6) valor do pagamento, 7) tipo do pagamento, 8) cidade do vendedor, 9) latitude e longitude da cidade do vendedor.**

In [20]:
query = """
SELECT 
    o.order_id,
    c.customer_state,
    c.customer_city, 
    o.order_status,
    p.product_category_name,
    oi.price,
    s.seller_city,
    s.seller_state,
    o.order_approved_at,
    op.payment_type,
    or2.review_score 
FROM orders o INNER JOIN order_items oi ON (o.order_id = oi.order_id)
              INNER JOIN customer c ON (o.customer_id = c.customer_id)
              INNER JOIN products p ON (oi.product_id = p.product_id)
              INNER JOIN sellers s ON (oi.seller_id = s.seller_id)
              INNER JOIN order_payments op ON (o.order_id = op.order_id)
              INNER JOIN order_reviews or2 ON (o.order_id = or2.order_id)
WHERE or2.review_score = 1
LIMIT 70;
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,order_id,customer_state,customer_city,order_status,product_category_name,price,seller_city,seller_state,order_approved_at,payment_type,review_score
0,b18dcdf73be66366873cd26c5724d1dc,SP,sao paulo,delivered,cama_mesa_banho,119.0,belo horizonte,MG,2018-04-09 20:10:35,credit_card,1
1,b18dcdf73be66366873cd26c5724d1dc,SP,sao paulo,delivered,cama_mesa_banho,99.9,piracicaba,SP,2018-04-09 20:10:35,credit_card,1
2,b18dcdf73be66366873cd26c5724d1dc,SP,sao paulo,delivered,cama_mesa_banho,99.9,piracicaba,SP,2018-04-09 20:10:35,credit_card,1
3,b18dcdf73be66366873cd26c5724d1dc,SP,sao paulo,delivered,cama_mesa_banho,99.9,piracicaba,SP,2018-04-09 20:10:35,credit_card,1
4,583174fbe37d3d5f0d6661be3aad1786,SP,guarulhos,canceled,malas_acessorios,89.9,sao paulo,SP,2018-08-05 19:24:33,credit_card,1


**17. Quantos tipos de pagamentos foram usados pelo cliente para pagar o pedido 'e481f51cbdc54678b7cc49136f2d6af7’**

In [21]:
query = """
SELECT 
    COUNT(op.payment_type) 
FROM orders o LEFT JOIN order_payments op ON (op.order_id = o.order_id)
WHERE O.order_id = 'e481f51cbdc54678b7cc49136f2d6af7'
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,COUNT(op.payment_type)
0,3


**18. Quantos pedidos (orders) não tem nenhuma avaliação (review) ?**

In [22]:
query = """
SELECT
    COUNT(or2.review_id) 
FROM orders o LEFT JOIN order_reviews or2 ON ( or2.order_id = o.order_id )
WHERE or2.review_id ISNULL
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,COUNT(or2.review_id)
0,0


**19. Quais são os top 10 vendedores com mais clientes?**

In [23]:
query = """
SELECT 
    s.seller_id,
    COUNT(DISTINCT c.customer_id) 
FROM sellers s LEFT JOIN order_items oi ON (s.seller_id = oi.seller_id)
               LEFT JOIN orders o ON (o.order_id = oi.order_id)
               LEFT JOIN customer c ON (c.customer_id = o.customer_id)
GROUP BY s.seller_id 
ORDER BY COUNT(DISTINCT c.customer_id) DESC
LIMIT 10;
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,seller_id,COUNT(DISTINCT c.customer_id)
0,6560211a19b47992c3666cc44a7e94c0,1854
1,4a3ca9315b744ce9f8e9374361493884,1806
2,cc419e0650a3c5ba77189a1882b7556a,1706
3,1f50f920176fa81dab994f9023523100,1404
4,da8622b14eb17ae2831f4ac5b9dab84a,1314


**20. Qual o número de pedido com o tipo de pagamento igual a “boleto”?**

In [24]:
query = """
SELECT
    COUNT(o.order_id) 
FROM orders o 
WHERE o.order_id IN (SELECT DISTINCT op.order_id 
                     FROM order_payments op
                     WHERE op.payment_type = 'boleto')
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,COUNT(o.order_id)
0,19784


**21. Refaça o exercício 01 usando união de tabelas.**

In [25]:
query = """
SELECT 
    COUNT(o.order_id)
 FROM orders o LEFT JOIN order_payments op ON (o.order_id = op.order_id)
 WHERE op.payment_type = 'boleto'
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,COUNT(o.order_id)
0,19784


**22. Cria uma tabela que mostre a média de avaliações por dia, a média de preço por dia, a soma dos preços por dia, o preço mínimo por dia, o número de pedidos por dia e o número de clientes únicos que compraram no dia.**

In [28]:
query = """
SELECT 
    t1.data_,
    t1.avg_reviews,
    t2.avg_price,
    t2.sum_price,
    t2.min_price,
    t3.num_customer
FROM (SELECT 
        DATE(or2.review_creation_date) as data_,
        AVG(or2.review_score) as avg_reviews
      FROM order_reviews or2 
      GROUP BY DATE(or2.review_creation_date)) as t1 LEFT JOIN (SELECT 
                                                                    DATE(oi.shipping_limit_date) as data_,
                                                                    AVG(oi.price) as avg_price,
                                                                    SUM(oi.price) as sum_price,
                                                                    MIN(oi.price) as min_price
                                                                FROM order_items oi 
                                                                GROUP BY DATE(oi.shipping_limit_date)) as t2 ON (t1.data_ = t2.data_)LEFT JOIN (SELECT 
                                                                                DATE(o.order_approved_at) as data_,
                                                                                COUNT(o.customer_id) as num_customer
                                                                              FROM orders o 
                                                                              GROUP BY DATE(o.order_approved_at)) as t3 ON (t2.data_ = t3.data_)
"""

cur.execute(query)

# como a biblioteca pandas, o resultado da consulta está em dataframe
df = pd.read_sql(query, con = con)
df.head()

,data_,avg_reviews,avg_price,sum_price,min_price,num_customer
0,2016-10-02,1.0,NaN,NaN,NaN,NaN
1,2016-10-06,1.0,NaN,NaN,NaN,NaN
2,2016-10-09,1.0,128.797500,1545.57,22.37,10.0
3,2016-10-15,4.0,92.313846,2400.16,6.00,NaN
4,2016-10-16,5.0,276.431667,1658.59,24.90,NaN
